## 引入相關函式庫
```python
from api import bingx as api_entry
```
此行依據你的交易所引入相對應的 api 套件

In [1]:
# Importing required libraries
import talib as ta
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np
from api import bingx as api_entry

import pandas as pd
from datetime import datetime

Successfully read secret key from .env\api_key.json
Simulation mode is ON


## 獲取交易所 api 回傳的 K 線數據
獲取數據後會進行處理，處理內容有：
1. 順序反轉(時間變為由前到後排序)
2. 將原本的 Time Stamp 轉換為 datetime
3. 製作一個 `pd.DataFrame` ，儲存 K 線數據並用於後續圖表顯現及 `talib` 指標計算 (`kLine_df`)

In [2]:
# 獲取K線數據(反轉成正序)

kLines = api_entry.get_kLine_data(symbol='BTC-USDT', interval='15m', limit=1000)
kLines.reverse()
print(kLines[:2])

[{'open': '83981.2', 'close': '84128.6', 'high': '84171.4', 'low': '83981.0', 'volume': '386.50', 'time': 1744865100000}, {'open': '84128.4', 'close': '84196.5', 'high': '84243.7', 'low': '84058.5', 'volume': '228.01', 'time': 1744866000000}]


In [3]:
kLine_df = pd.DataFrame(kLines)

# 將字串欄位轉成 float
for col in ['open', 'close', 'high', 'low', 'volume']:
    kLine_df[col] = kLine_df[col].astype(float)

# 將時間戳(毫秒)轉成 datetime
kLine_df['time'] = pd.to_datetime(kLine_df['time'], unit='ms')

In [4]:
# 繪製 Plotly K 線圖

fig = go.Figure(data=[go.Candlestick(
    x=kLine_df['time'],
    open=kLine_df['open'],
    high=kLine_df['high'],
    low=kLine_df['low'],
    close=kLine_df['close'],
    increasing_line_color='green',
    decreasing_line_color='red'
)])

fig.update_layout(
    title='Candlestick chart',
    xaxis_title='Time',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False
)

fig.show()

## 由上面的 K 線數據計算 RSI
使用 `kLine_df` 的 close value ，以14個時間週期計算 RSI，儲存在 `kLine_df` 的 "RSI" Column 中

In [5]:
# Calculate the RSA Value using TA-Lib

# Access the Close price using the MultiIndex
close_values = kLine_df['close'].values

kLine_df['RSI'] = ta.RSI(close_values, timeperiod=14)
kLine_df.head(20)  # Display the first 20 rows of the DataFrame

,open,close,high,low,volume,time,RSI
0,83981.2,84128.6,84171.4,83981.0,386.50,2025-04-17 04:45:00,NaN
1,84128.4,84196.5,84243.7,84058.5,228.01,2025-04-17 05:00:00,NaN
2,84194.3,84249.4,84326.0,84182.7,97.22,2025-04-17 05:15:00,NaN
3,84249.4,84376.4,84432.6,84243.0,133.33,2025-04-17 05:30:00,NaN
4,84376.6,84362.4,84469.2,84319.3,120.14,2025-04-17 05:45:00,NaN
5,84362.6,84400.2,84488.0,84282.4,78.89,2025-04-17 06:00:00,NaN
6,84404.1,84575.2,84647.8,84336.8,109.35,2025-04-17 06:15:00,NaN
7,84575.0,84732.1,84795.0,84563.9,334.52,2025-04-17 06:30:00,NaN
8,84732.3,84756.8,84845.9,84681.0,110.80,2025-04-17 06:45:00,NaN
9,84759.7,84649.8,84832.5,84614.5,158.05,2025-04-17 07:00:00,NaN


In [6]:
# Draw the RSI line chart using Plotly, with k-Line chart above and RSI chart below

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.1,
                    subplot_titles=('Candlestick Chart', 'RSI Chart'),
                    row_width=[0.2, 0.7])  # Set the height ratio
# Add the candlestick chart to the first row
fig.add_trace(go.Candlestick(
    x=kLine_df['time'],
    open=kLine_df['open'],
    high=kLine_df['high'],
    low=kLine_df['low'],
    close=kLine_df['close'],
    increasing_line_color='green',
    decreasing_line_color='red'
), row=1, col=1)
# Add the RSI line chart to the second row
fig.add_trace(go.Scatter(
    x=kLine_df['time'],
    y=kLine_df['RSI'],
    mode='lines',
    name='RSI',
    line=dict(color='blue', width=2)
), row=2, col=1)
# Update the layout of the figure
fig.update_layout(
    title='Candlestick and RSI Chart',
    xaxis_title='Time',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False,
    height=800  # Set the height of the entire figure
)
fig.show()

## 由上面的 K 線數據計算 EMA
計算 3 個 EMA 數值
1. EMA20
2. EMA40
3. EMA60

In [7]:
# Calculate the EMA Values

# 計算 EMA
kLine_df['EMA20'] = kLine_df['close'].ewm(span=20, adjust=False).mean()
kLine_df['EMA40'] = kLine_df['close'].ewm(span=40, adjust=False).mean()
kLine_df['EMA60'] = kLine_df['close'].ewm(span=60, adjust=False).mean()

In [8]:
# 繪製 K 線圖和 EMA 線

fig = go.Figure()

# K 線圖
fig.add_trace(go.Candlestick(
    x=kLine_df['time'],
    open=kLine_df['open'],
    high=kLine_df['high'],
    low=kLine_df['low'],
    close=kLine_df['close'],
    increasing_line_color='green',
    decreasing_line_color='red',
    name='Candlestick'
))

# EMA 線
fig.add_trace(go.Scatter(
    x=kLine_df['time'],
    y=kLine_df['EMA20'],
    mode='lines',
    line=dict(color='blue', width=1),
    name='EMA20'
))
fig.add_trace(go.Scatter(
    x=kLine_df['time'],
    y=kLine_df['EMA40'],
    mode='lines',
    line=dict(color='orange', width=1),
    name='EMA40'
))
fig.add_trace(go.Scatter(
    x=kLine_df['time'],
    y=kLine_df['EMA60'],
    mode='lines',
    line=dict(color='purple', width=1),
    name='EMA60'
))

fig.update_layout(
    title='Candlestick chart with EMA20, EMA40, EMA60',
    xaxis_title='Time',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False
)

fig.show()